## YOLOv8-Pose and Action Prediction:

### Check for GPU:

In [ ]:
!nvidia-smi

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

## Download Demo Video:

In [ ]:
## 10 sec video
!gdown https://drive.google.com/uc?id=1t0EctgtEbiW5WEv6gZkTMw2TjQlDjkry

## Download Requirements:

In [ ]:
!pip install ultralytics==8.0.93
!pip install gdown
!pip install imageio[ffmpeg]
!pip install imageio[pyav]

## Download Model Files:

In [ ]:
## YOLOv8-custom.pt model.
!wget https://github.com/ultralytics/assets/releases/download/v0.0.0/yolov8x-pose.pt
## YOLOv8x-pose-p6.pt model for pose estimation.
!gdown https://drive.google.com/uc?id=1CJjmDetBADOd6DUIPgSgZUghqEGF3EcS

In [ ]:
import math

def calculate_angle(joint_a, joint_b, image_width, image_height):

    norm_joint_a = [joint_a[0] / image_width, joint_a[1] / image_height, joint_a[2]]
    norm_joint_b = [joint_b[0] / image_width, joint_b[1] / image_height, joint_b[2]]

    vector = [norm_joint_b[0] - norm_joint_a[0], norm_joint_b[1] - norm_joint_a[1], norm_joint_b[2] - norm_joint_a[2]]
    
    magnitude = math.sqrt(vector[0] ** 2 + vector[1] ** 2 + vector[2] ** 2)
    
    angle_rad = math.acos(vector[2] / magnitude)
    
    angle_deg = math.degrees(angle_rad)
    
    return angle_deg

In [ ]:
def calculate_distance(joint_a, joint_b, image_width, image_height):
    norm_joint_a = [joint_a[0] / image_width, joint_a[1] / image_height, joint_a[2]]
    norm_joint_b = [joint_b[0] / image_width, joint_b[1] / image_height, joint_b[2]]

    distance = math.sqrt((norm_joint_b[0] - norm_joint_a[0])**2 + (norm_joint_b[1] - norm_joint_a[1])**2 + (norm_joint_b[2] - norm_joint_a[2])**2)
    
    return distance

## Run Inference:

In [ ]:
import cv2
from ultralytics import YOLO
from google.colab.patches import cv2_imshow

model = YOLO('yolov8x_custom.pt')
model_pose = YOLO('yolov8x-pose-p6.pt')

video_path = "quality.mp4"
cap = cv2.VideoCapture(video_path)

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

output_path = "output.mp4"

fourcc = cv2.VideoWriter_fourcc(*'mp4v')

out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

while cap.isOpened():
    success, frame = cap.read()

    if success:
        results_boxes = model.predict(frame, save=True, imgsz=1920, conf=0.5)
        results_poses = model_pose.predict(frame, save=True, imgsz=1920, conf=0.001)

        n = len(results_boxes[0])
        i = 0
        for keypoint in results_poses[0].keypoints:

          joint_right_foot = keypoint[16].tolist()
          joint_left_foot = keypoint[15].tolist()
          joint_right_knee = keypoint[14].tolist()
          joint_left_knee = keypoint[13].tolist()
          joint_right_arm = keypoint[10].tolist()
          joint_left_arm = keypoint[6].tolist()
          joint_right_shoulder = keypoint[9].tolist()
          joint_left_shoulder = keypoint[5].tolist()
          joint_right_hip = keypoint[12].tolist()
          joint_left_hip = keypoint[11].tolist()


          angle_knee = calculate_angle(joint_right_knee, joint_left_knee, frame.shape[1], frame.shape[0])
          angle_kick_right = calculate_angle(joint_right_foot, joint_right_hip, frame.shape[1], frame.shape[0])
          angle_kick_left = calculate_angle(joint_left_foot, joint_left_hip, frame.shape[1], frame.shape[0])
          angle_left_arm = calculate_angle(joint_left_shoulder, joint_left_arm, frame.shape[1], frame.shape[0])
          angle_right_arm = calculate_angle(joint_right_shoulder, joint_left_shoulder, frame.shape[1], frame.shape[0])

          distance_knee = calculate_distance(joint_left_knee, joint_right_knee, frame.shape[1], frame.shape[0])
          distance_left_palm = calculate_distance(joint_left_arm, joint_left_shoulder, frame.shape[1], frame.shape[0])
          distance_right_palm = calculate_distance(joint_right_arm, joint_right_shoulder, frame.shape[1], frame.shape[0])

          
          if distance_knee >= 0.01 and (abs(angle_left_arm) >= 20 or abs(angle_right_arm) >= 20):
            print(f"Player on x:{results_boxes[0].boxes[i].xyxy.cpu().tolist()[0][0]} y:{results_boxes[0].boxes[i].xyxy.cpu().tolist()[0][1]} is running: ")
          elif abs(angle_kick_left) >= 80 or abs(angle_kick_right) >= 80:
            print(f"Player on x:{results_boxes[0].boxes[i].xyxy.cpu().tolist()[0][0]} y:{results_boxes[0].boxes[i].xyxy.cpu().tolist()[0][1]} is kicking: ")
          else:
            print(f"Player on x:{results_boxes[0].boxes[i].xyxy.cpu().tolist()[0][0]} y:{results_boxes[0].boxes[i].xyxy.cpu().tolist()[0][1]} is standing or doing nothing: ")
          i += 1
          if i == n:
            break
        
        annotated_frame_poses = results_poses[0].plot(boxes=False)
        annotated_frame_boxes = results_boxes[0].plot()
        merged_frame = cv2.addWeighted(annotated_frame_boxes, 0.5, annotated_frame_poses, 0.5, 0)

        out.write(merged_frame)

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        break

out.release()
cap.release()
cv2.destroyAllWindows()

## Display Video:

In [ ]:
import imageio
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

def display_video(video):
    fig = plt.figure(figsize=(8,8))

    mov = []
    for i in range(len(video)): 
        img = plt.imshow(video[i], animated=True)
        plt.axis('off')
        mov.append([img])

    anime = animation.ArtistAnimation(fig, mov, interval=50, repeat_delay=1000)

    plt.close()
    return anime

In [ ]:
video = imageio.mimread(f'./runs/track/exp/quality.mp4', memtest=False)
HTML(display_video(video).to_html5_video())